In [10]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
train_data = pd.read_csv("../data/train/train.csv")
test_data = pd.read_csv("../data/test/test.csv")

In [12]:
X = train_data[['Human', 'Near','Occlusion']].values
y = train_data['Pawpularity'].values

In [13]:
# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

In [14]:
# Create datasets and dataloaders
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [15]:
# Define the neural network
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Linear(3, 32)  # Only three features
        self.layer2 = nn.Linear(32, 16)
        self.output_layer = nn.Linear(16, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.output_layer(x)
        return x

model = SimpleNet()

In [16]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets.view(-1, 1))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

KeyboardInterrupt: 

In [ ]:
model.eval()
with torch.no_grad():
    val_loss = sum(criterion(model(inputs), targets.view(-1, 1)).item() for inputs, targets in val_loader) / len(val_loader)
print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}')


Epoch [100/100], Training Loss: 324.5054, Validation Loss: 442.4454


In [ ]:
# Print predicted Pawpularity scores
model.eval()
predictions = []
with torch.no_grad():
    for inputs, targets in val_loader:
        outputs = model(inputs)
        predictions.extend(outputs.view(-1).tolist())

# Print predictions
print("Predicted Pawpularity Scores:")
print(predictions)
print(f"Total Predictions Made: {len(predictions)}")

Predicted Pawpularity Scores:
[37.67961120605469, 37.412696838378906, 37.67961120605469, 37.67961120605469, 37.67961120605469, 37.412696838378906, 37.67961120605469, 37.67961120605469, 37.67961120605469, 37.67961120605469, 37.44871520996094, 37.67961120605469, 37.412696838378906, 37.67961120605469, 37.67961120605469, 37.67961120605469, 37.67961120605469, 37.412696838378906, 37.67961120605469, 37.67961120605469, 37.67961120605469, 37.67961120605469, 37.67961120605469, 37.67961120605469, 37.67961120605469, 37.412696838378906, 36.751346588134766, 37.27255630493164, 37.412696838378906, 37.67961120605469, 37.67961120605469, 37.67961120605469, 37.72829818725586, 37.67961120605469, 37.67961120605469, 37.67961120605469, 37.67961120605469, 38.15602111816406, 36.751346588134766, 37.67961120605469, 37.67961120605469, 37.67961120605469, 37.67961120605469, 36.751346588134766, 37.67961120605469, 37.67961120605469, 37.412696838378906, 37.412696838378906, 37.67961120605469, 37.67961120605469, 37.67961